<a href="https://colab.research.google.com/github/urmilapol/urmilapolprojects/blob/master/Copy_of_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Connect to the Postgres database we set up earlier
# (Assuming your connection object is named pg_conn)
query = "SELECT user_id, action, value FROM silver_events LIMIT 10;"

try:
    df = pd.read_sql(query, pg_conn)
    print(f"Total Records in Database: {pd.read_sql('SELECT COUNT(*) FROM silver_events', pg_conn).iloc[0,0]}")
    print("\n--- Last 10 Live Events (showing user_id, action, value) ---")
    print(df)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the Consumer has created the 'silver_events' table first!")

Error: Execution failed on sql 'SELECT u_id, act, val FROM silver_events LIMIT 10;': column "u_id" does not exist
LINE 1: SELECT u_id, act, val FROM silver_events LIMIT 10;
               ^

Make sure the Consumer has created the 'silver_events' table first!


/tmp/ipython-input-3831125984.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pg_conn)


In [ ]:
import pandas as pd

# Connect to the Postgres database we set up earlier
# (Assuming your connection object is named pg_conn)
query = "SELECT user_id, action, value FROM silver_events LIMIT 10;"

try:
    df = pd.read_sql(query, pg_conn)
    print(f"Total Records in Database: {pd.read_sql('SELECT COUNT(*) FROM silver_events', pg_conn).iloc[0,0]}")
    print("\n--- Last 10 Live Events (showing user_id, action, value) ---")
    print(df)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the Consumer has created the 'silver_events' table first!")

Error: Execution failed on sql 'SELECT u_id, act, val FROM silver_events LIMIT 10;': column "u_id" does not exist
LINE 1: SELECT u_id, act, val FROM silver_events LIMIT 10;
               ^

Make sure the Consumer has created the 'silver_events' table first!


/tmp/ipython-input-3831125984.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pg_conn)


In [ ]:
import pandas as pd

# Connect to the Postgres database we set up earlier
# (Assuming your connection object is named pg_conn)
query = "SELECT user_id, action, value FROM silver_events LIMIT 10;"

try:
    df = pd.read_sql(query, pg_conn)
    print(f"Total Records in Database: {pd.read_sql('SELECT COUNT(*) FROM silver_events', pg_conn).iloc[0,0]}")
    print("\n--- Last 10 Live Events (showing user_id, action, value) ---")
    print(df)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the Consumer has created the 'silver_events' table first!")

Error: Execution failed on sql 'SELECT u_id, act, val FROM silver_events LIMIT 10;': column "u_id" does not exist
LINE 1: SELECT u_id, act, val FROM silver_events LIMIT 10;
               ^

Make sure the Consumer has created the 'silver_events' table first!


/tmp/ipython-input-3831125984.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pg_conn)


# **Real-Time Data Streaming**

Adding Apache Kafka to this lab transforms it from a simple batch ETL process into a modern Event-Driven Data Pipeline. In this scenario, we move away from "pulling" data and toward "streaming" data as it happens.

The Streaming Architecture
In a real-world data engineering environment:

Producer: A web app sends user events to a Kafka Topic.

Stream Processor: A Python script listens to the topic, cleans the data in real-time.

Sink (Storage): The processed data is written to PostgreSQL for immediate dashboarding.

In [ ]:
# 1. Install Postgres
!apt install postgresql postgresql-contrib &> /dev/null

# Configure PostgreSQL to accept password authentication for user 'postgres' and create database
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'password123';"
!sudo -u postgres psql -c "CREATE DATABASE silver_events;"
!service postgresql restart

 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE
ERROR:  database "silver_events" already exists
 * Restarting PostgreSQL 14 database server
   ...done.


In [ ]:
# 2. Install Kafka (Downloads the binaries)
!curl -sSOL https://archive.apache.org/dist/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz

# 3. Start Zookeeper and Kafka Server
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh ./kafka_2.13-3.1.0/config/zookeeper.properties > zookeeper.log 2>&1 &
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh ./kafka_2.13-3.1.0/config/server.properties > kafka.log 2>&1 &

# Give Kafka a moment to start up
!sleep 10

# **This script simulates a continuous stream of user clickstream data.**

In [ ]:
import threading
import json
import time
import random
from kafka import KafkaProducer # Moved this import here

!pip install kafka-python # Moved installation to the top level

def run_producer():
    # The entire producer logic will now be correctly placed and indented here
    producer = KafkaProducer(
        bootstrap_servers=['localhost:9092'],
        value_serializer=lambda x: json.dumps(x).encode('utf-8')
    )

    actions = ['click', 'view', 'add_to_cart', 'purchase']

    print("Streaming started from producer thread...")
    while True:
        data = {
            "user_id": random.randint(1000, 9999),
            "action": random.choice(actions),
            "timestamp": time.time()
        }
        producer.send('user-events', value=data)
        time.sleep(1)  # Send one event per second

# This starts the producer in the background
producer_thread = threading.Thread(target=run_producer, daemon=True)
producer_thread.start()
print("Producer is running in the background...")

Producer is running in the background...


# This script acts as the Data Engineer's "Processor." It consumes the raw stream, filters out only the "purchase" events, and retrieves them into PostgreSQL.


In [ ]:
from kafka import KafkaConsumer
import psycopg2
import json

# Connect to Postgres
conn = psycopg2.connect("host=localhost dbname=silver_events user=postgres password=password123")
cur = conn.cursor()

# Create the table if it doesn't exist
cur.execute("CREATE TABLE IF NOT EXISTS silver_events (u_id INTEGER, act VARCHAR(255), val INTEGER);")
conn.commit()

# Initialize Kafka Consumer
consumer = KafkaConsumer(
    'user-events',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Listening for events...")
for message in consumer:
    event = message.value

    # Logic: Only store high-value events (Purchases)
    if event['action'] == 'purchase':
        print(f"Storing Purchase: {event}")
        cur.execute(
            "INSERT INTO silver_events (u_id, act, val) VALUES (%s, %s, %s)",
            (event['user_id'], event['action'], 1)
        )
        conn.commit()

Listening for events...
Storing Purchase: {'user_id': 7044, 'action': 'purchase', 'timestamp': 1767784121.754121}
Storing Purchase: {'user_id': 4352, 'action': 'purchase', 'timestamp': 1767784131.758446}
Storing Purchase: {'user_id': 5594, 'action': 'purchase', 'timestamp': 1767784135.7601395}
Storing Purchase: {'user_id': 1404, 'action': 'purchase', 'timestamp': 1767784140.7622867}
Storing Purchase: {'user_id': 5848, 'action': 'purchase', 'timestamp': 1767784141.7627509}
Storing Purchase: {'user_id': 8297, 'action': 'purchase', 'timestamp': 1767784145.7642791}
Storing Purchase: {'user_id': 7883, 'action': 'purchase', 'timestamp': 1767784146.7646616}
Storing Purchase: {'user_id': 8405, 'action': 'purchase', 'timestamp': 1767784148.7655537}
Storing Purchase: {'user_id': 6585, 'action': 'purchase', 'timestamp': 1767784151.7667046}
Storing Purchase: {'user_id': 6901, 'action': 'purchase', 'timestamp': 1767784152.767161}
Storing Purchase: {'user_id': 5277, 'action': 'purchase', 'timestamp'

**SQL command to verify how many rows have successfully moved from the Producer -> Kafka -> Consumer -> Postgres?**
Step 1: Initialize the Connection
Before running the verification script, ensure you have run this "Setup" cell to establish the connection:


In [7]:
import psycopg2

# Re-establish connection to local Postgres in Colab
try:
    pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="password123", # Use the password you set in the install step
        host="127.0.0.1",
        port="5432"
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection failed: connection to server at "127.0.0.1", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?



In [8]:
print("Restarting PostgreSQL service...")
!service postgresql restart
!service postgresql status
print("PostgreSQL service status checked. Now attempting to reconnect.")

Restarting PostgreSQL service...
postgresql: unrecognized service
postgresql: unrecognized service
PostgreSQL service status checked. Now attempting to reconnect.


In [9]:
# 1. Install Postgres (if not already installed)
!apt install postgresql postgresql-contrib &> /dev/null

# Configure PostgreSQL to accept password authentication for user 'postgres' and create database
# These commands will start the service if it's not already running, and restart it to apply changes.
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'password123';"
!sudo -u postgres psql -c "CREATE DATABASE silver_events;" 2>/dev/null || true # Ignore error if database exists
!service postgresql restart

import psycopg2

# Attempt to connect to local Postgres
try:
    pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="password123",
        host="127.0.0.1",
        port="5432"
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE
CREATE DATABASE
 * Restarting PostgreSQL 14 database server
   ...done.
Connection successful!


In [10]:
# 1. Install Postgres (if not already installed)
!apt install postgresql postgresql-contrib &> /dev/null

# Configure PostgreSQL to accept password authentication for user 'postgres' and create database
# These commands will start the service if it's not already running, and restart it to apply changes.
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'password123';"
!sudo -u postgres psql -c "CREATE DATABASE silver_events;" 2>/dev/null || true # Ignore error if database exists
!service postgresql restart

import psycopg2

# Attempt to connect to local Postgres
try:
    pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="password123",
        host="127.0.0.1",
        port="5432"
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE
 * Restarting PostgreSQL 14 database server
   ...done.
Connection successful!


In [11]:
import psycopg2

# Re-establish connection to local Postgres in Colab
try:
    pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="password123", # Use the password you set in the install step
        host="127.0.0.1",
        port="5432"
    )
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


Step 2: Fix the "Schema" (If table is missing)
The Consumer script is supposed to create the table. If it hasn't, the pd.read_sql will fail. You can manually create it to be safe:

In [12]:
cur = pg_conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS silver_events (
        user_id INT,
        action TEXT,
        value INT,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
""")
pg_conn.commit()
print("Table 'silver_events' is ready.")

Table 'silver_events' is ready.


In [15]:
import pandas as pd

# Connect to the Postgres database we set up earlier
# (Assuming your connection object is named pg_conn)
query = "SELECT user_id, action, value FROM silver_events LIMIT 10;"

try:
    df = pd.read_sql(query, pg_conn)
    print(f"Total Records in Database: {pd.read_sql('SELECT COUNT(*) FROM silver_events', pg_conn).iloc[0,0]}")
    print("\n--- Last 10 Live Events (showing user_id, action, value) ---")
    print(df)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure the Consumer has created the 'silver_events' table first!")

Total Records in Database: 0

--- Last 10 Live Events (showing user_id, action, value) ---
Empty DataFrame
Columns: [user_id, action, value]
Index: []


/tmp/ipython-input-1760815732.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, pg_conn)
/tmp/ipython-input-1760815732.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(f"Total Records in Database: {pd.read_sql('SELECT COUNT(*) FROM silver_events', pg_conn).iloc[0,0]}")
